In [1]:
import svmnbc
import sys
from PyQt5 import QtWidgets, uic

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anjas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
app=QtWidgets.QApplication([])
call=uic.loadUi("gui.ui")
call.show()

In [3]:
def hasil(): 
    banyak_training=float(call.in_training.currentText())/100
    asd=float(banyak_training)

    in_kernel = call.in_kernel.currentText()
    
    svm = svmnbc.Analis(asd).svm(in_kernel)
    nbc = svmnbc.Analis(asd).nbc()

    call.svm_a.setText(str(svm[0]))
    call.svm_r.setText(str(svm[1]))
    call.svm_p.setText(str(svm[2]))
    call.nbc_a.setText(str(nbc[0]))
    call.nbc_r.setText(str(nbc[1]))
    call.nbc_p.setText(str(nbc[2]))

In [4]:
call.ok.clicked.connect(hasil)
app.exec()

0